# Building & Deploying an H2O Model

**Purpose:** This notebook is designed to interactively guide the user through an end-to-end process for deploying a machine learning workflow utilizing h2o.ai's Flow browser-based interface.  It provides a generic dataset, but the notebook can be repurposed for any structured dataset (.csv and .xlsx-formats).

## The dataset
Kickstarter is one of the main online crowdfunding platforms in the world. The dataset provided contains data on tens of thousands of projects launched on the platform in 2018. The datasets provided have the same structure and contain the following columns:

- **ID**: internal ID, _numeric_
- **name**: name of the project, _string_
- **category**: project's category, _string_
- **main_category**: campaign's category, _string_
- **currency**: project's currency, _string_
- **deadline**: project's deadline date, _timestamp_
- **goal**: fundraising goal, _numeric_
- **launched**: project's start date, _timestamp_
- **pledged**: amount pledged by backers (project's currency), _numeric_
- **state**: project's current state, _string_; **this is what you have to predict**
- **backers**: amount of poeple that backed the project, _numeric_
- **country**: project's country, _string_
- **usd pledged**: amount pledged by backers converted to USD (conversion made by KS), _numeric_
- **usd_pledged_real**: amount pledged by backers converted to USD (conversion made by fixer.io api), _numeric_
- **usd_goal_real**: fundraising goal is USD, _numeric_
- **launch_month**: the numeric value for the month of the year in which the project was launched, _numeric_
- **launch_dow**: an integer value ranging from 1 (Sunday) to 7 (Saturday) b, _numeric_
- **duration**: the number of days between the launched date and deadline date, _numeric_

</br></br>

**Resources**:
* https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html
    
## Table of Contents

**1.0** **- Ingest Data**
    * 1.1 - Set Your Working Directory
    * 1.2 - Upload Your Data (for Modeling)
    * 1.3 - Select a Data Frame (for Modeling)
     
**2.0** **- Initiate H2O**
    * 2.1 - Select Your Target Variable
    * 2.2 - Initiate H2O
    * 2.3 - Configure Models in H2O Flow

**3.0** **- Score New Data**
    * 3.1 - Upload Your Data (for Scoring)
    * 3.2 - Select a Data Frame (for Scoring)
    * 3.3 - Perform Singe or Batch Scoring

## Dependencies

This script was executed using the following version of Python:
* **Python 3.6.2 :: Anaconda, Inc.**

Use this link to install Python on your machine:
* https://www.anaconda.com/distribution/#download-section

**About Python Versions:**
If you are running a higher-version of Python and this notebook fails to execute properly, you can downgrade your version in the terminal by running the following commands:
* conda search python [to see which versions are available on your machine]
* conda install python=3.6.2 [which will switch the active version to 3.6.2; if available in the list above]

**About Python Packages:**
All packages used in this notebook can be installed on your machine using the "pip install [package_name]" command on your terminal.  Be sure you've installed each of the packages below before attempting to execute the notebook.

Current package requirements include:
* os - https://docs.python.org/3/library/os.html
* Pandas - https://pandas.pydata.org/
* Datetime - https://docs.python.org/3/library/datetime.html
* re - https://docs.python.org/3/library/re.html
* Numpy - http://www.numpy.org/
* ipywidgets - https://ipywidgets.readthedocs.io/en/stable/user_install.html
* ipython - https://ipython.org/ipython-doc/rel-0.10.2/html/interactive/extension_api.html
* h2o - http://docs.h2o.ai/h2o/latest-stable/h2o-docs/downloading.html#install-in-python
* scikit-learn - https://scikit-learn.org/stable/install.html
* requests - https://2.python-requests.org/en/master/user/install/
* io - https://docs.python.org/3/library/io.html
* warnings - https://docs.python.org/3/library/warnings.html
* json - https://docs.python.org/3/library/json.html
* subprocess - https://docs.python.org/3/library/subprocess.html

The current template uses the following versions:
* os== module 'os' from '/anaconda3/lib/python3.6/os.py'
* pandas==0.24.1
* datetime== module 'datetime' from '/anaconda3/lib/python3.6/datetime.py'
* re== module 're' from '/anaconda3/lib/python3.6/re.py'
* numpy==1.16.1
* ipywidgets==7.4.2
* ipython==6.2.1
* h2o==3.24.0.4
* scikit-learn==0.19.1
* requests==2.18.4
* io== module 'io' from '/anaconda3/lib/python3.6/io.py'
* warnings== module 'warnings' from '/anaconda3/lib/python3.6/warnings.py'
* json

## Before you begin, ensure you've installed the required Python packages

* See the list above and make note of the specific versions that were used in this notebook

In [2]:
############################################
###### Import required Python packages #####
############################################

import os
import pandas as pd
import re
import datetime as dt
from datetime import timezone
import numpy as np
from ipywidgets import interact, interactive, IntSlider, Layout
import ipywidgets as widgets
from IPython.display import display
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
import io
import requests
import subprocess
import json
import warnings
warnings.filterwarnings('ignore')

## Note: Code Cells are Hidden by Default for Ease-of-Use

This notebook incorporates interactive "widgets" which will result in large blocks of cells being utilized to enable specific user-interactions.  Executing this cell will hide all "Code" cells while making all outputs visible to the user.  Refer to the link below for the source or simply "run" the block below to see the impact on the rest of the notebook.

* https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer

#### Disclaimer:
* As the "output text" notes, simply click the "here" hyperlink in the text to toggle on/off this feature

In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 1.0 - Data Ingestion

The series of code blocks below will walk you through the process of mapping to your working directory and uploading your dataset.

## 1.1 - Set Your Working Directory

Your "working directory" is a folder location on your computer that will store files either read-in or written-out by this script.  This code by default will return your current, active directory.  You can change this directory by typing in a specific path into the text box provided.

## AN IMPORTANT NOTE ABOUT INTERACTIVE WIDGETS

This notebook uses interactive widgets to help you make selections and inputs more conveniently.  As you work through this notebook, be sure to follow the steps below to ensure your selections are incorporated in the cells that follow:

#### 1. Run the cell containing the interactive widget(s) to bring them into view
#### 2. Apply your selections and/or inputs to the widgets that appear
#### 3. DO NOT rerun the cell as it will erase your selections and inputs
#### 4. To proceed, simply click on the next cell in the notebook, and Run it

<br/>

In [4]:
set_working_directory = widgets.Text(
    value=os.getcwd(),
    placeholder='/Users/bblanchard006/Desktop/SMU/QTW/Week 13',
    description='Directory:',
    disabled=False,
    layout=Layout(width='100%')
)

display(set_working_directory)

Text(value='/Users/bblanchard006/Desktop/SMU/QTW/Week 13', description='Directory:', layout=Layout(width='100%…

## Reminder: Do not rerun the cell above after applying your inputs

### Click on and Run this cell to proceed

After executing the cell above, you can leave the default directory or overwrite the text string that appears with your desired folder directory. **DO NOT execute the cell again after making your update.** The input above will be fed into the following code cell, where it will either successfully map to the new directory or notify you of an error.

In [5]:
try:
    os.chdir(set_working_directory.value)
    print('Changed directory to {}'.format(set_working_directory.value))
except Exception as e:
    print('Failed to change directory')
    print(e)

Changed directory to /Users/bblanchard006/Desktop/SMU/QTW/Week 13


## 1.2 - Upload Your Data (Excel and CSV files)

The function in the code cell below will find, ingest, and format both xlsx and csv files.  This is the dataset with "known" values which will be used to train your models.

In [6]:
########################################
##### Data Ingestion Functions
########################################

def compile_raw_data(filename, tab_names, subfolder, delimiter_char = ',', skip_rows = 0, file_ext = 'xlsx'):
    
    # Inputs: 
    ## filename = 'sample.csv' | 'sample.xlsx' - the filename in the directory (including the extension) 
    ## tab_names = None | ['Sheet1,'Sheet2'] - None for csv; [comma separated list of tab names] for xlsx
    ## subfolder = 'source_data' - string containing the name of a folder in the working directory
    ## delimiter_char = ',' | ';' - None for xlsx
    ## rows to skip = default 0 - Not used for csv; trims the user-defined number of rows from an xlsx
    ## file extension = csv | xlsx
    
    # Description: reads in the workbook; standardizes header names; 
    # Outputs: returns a dictionary of dataframes

    master_data = {}
    if subfolder:
        file_path = subfolder+'/{}'.format(filename)
    else:
        file_path = filename

    if file_ext == 'csv':
        tab_names = [re.sub('.csv','', filename)]

    for tab in tab_names:
        try:
            if file_ext == 'xlsx':
                dframe = pd.read_excel(file_path, tab, skip_rows)
            elif file_ext == 'csv' and delimiter_char == ',':
                dframe = pd.read_csv(file_path, header=0, delimiter=',')
            else:
                dframe = pd.read_csv(file_path, header=0, delimiter=';')
                
            sanitizer = {
                        '$':'USD',
                        '(':' ',
                        ')':' ',
                        '/':' ',
                        '-':' ',
                        ',':' ',
                        '.':' '
            }
                        
            for key, value in sanitizer.items():
                dframe.rename(columns=lambda x: x.replace(key, value), inplace=True)
                
            dframe.rename(columns=lambda x: x.strip(), inplace=True)
            dframe.rename(columns=lambda x: re.sub(' +','_', x), inplace=True)
            
            dframe.columns = map(str.lower, dframe.columns)
            
            master_data.update({tab:dframe})
        except Exception as e:
            master_data.update({tab:'Failed'})
    
    return master_data

The code blocks below enable conditional filtering to support multiple file types. Further instructions are provided below:

**Uploading csv files**

To upload a csv file, complete these steps:
1. Type in your filename along with the extension (ex. sample.csv)
2. Check the 'csv' radio-button
3. Is your file in the main directory or a sub-folder in the directory:
    * Select the "no" radio-button if your file is in your main directory
    * Select the "yes" radio-button to expose a text-box where you can type-in the name of your sub-folder
    
**Uploading xlsx files**

To upload an xlsx file, complete these steps:
1. Type in your filename along with the extension (ex. sample.xlsx)
2. Check the 'xlsx' radio-button
3. Type in the tab-names you'd like to ingest (comma-separated; Sheet1,Sheet2,Sheet3)
4. If the data in your file has leading rows, select how many rows to skip before ingesting the data (ex. if your data starts on Row 2 in the Excel-file, set the Skip Rows value to 1)
5. Is your file in the main directory or a sub-folder in the directory:
    * Select the "no" radio-button if your file is in your main directory
    * Select the "yes" radio-button to expose a text-box where you can type-in the name of your sub-folder

In [7]:
upload_type = widgets.RadioButtons(
    options=['local', 'url'],
    description='File Location:',
    disabled=False
)

upload_url = widgets.Text(
    value='https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv',
    placeholder='http://',
    description='URL:',
    disabled=False,
    layout=Layout(width='80%')
)
upload_filename = widgets.Text(
    value='training_data.csv',
    placeholder='Sample File.csv',
    description='File Name:',
    disabled=False,
    layout=Layout(width='50%')
)

file_type = widgets.RadioButtons(
    options=['csv', 'xlsx'],
    description='File Type:',
    disabled=False
)

tab_names = widgets.Text(
    value='Sheet1, Sheet2, Sheet3, etc',
    placeholder='ALL EMPLOYEES, PAST EMPLOYEES',
    description='Tab(s):',
    disabled=False,
    layout=Layout(width='50%')
)

subfolder_name = widgets.Text(
    value='source_data',
    placeholder='Subfolder name',
    description='Subfolder:',
    disabled=False,
    layout=Layout(width='50%')
)

subfolder = widgets.RadioButtons(
    options=['no','yes'],
    value='no',
    description='Subfolder:',
    disabled=False
)

skip_rows = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Skip Rows:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

delimiter = widgets.RadioButtons(
    options=[',',';'],
    value=',',
    description='Delimiter:',
    disabled=False
)

def text_field(x):
    if(x=='xlsx'):
        display(tab_names)
        tab_names.on_submit(tab_names)
        display(skip_rows)
    else:
        display(delimiter)
        print('Tab Names: Not needed for csv files')

def sub_folder(y):
    if(y=='yes'):
        display(subfolder_name)
        subfolder_name.on_submit(subfolder_name)
    else:
        print('Using {} folder'.format(os.getcwd()))

def file_location(z):
    if(z=='local'):
        display(upload_filename)
        i = widgets.interactive(text_field, x=file_type)
        display(i)
        p = widgets.interactive(sub_folder, y=subfolder)
        display(p)
    else:
        display(upload_url)
    
q = widgets.interactive(file_location, z=upload_type)

display(q)

interactive(children=(RadioButtons(description='File Location:', options=('local', 'url'), value='local'), Out…

## Reminder: Do not rerun the cell above after applying your inputs

### Click on and Run this cell to proceed

The following code cell will attempt to ingest the data you've selected in the widgets above:

**Note About xlsx Files** - Depending on the number of tabs and the size of the data on each tab, ingesting an xlsx file can take several minutes to execute.  If possible, it may be more efficient to break your Excel file into separate csv files which take only a fraction of a second to ingest.

In [8]:
master_data = {}

if upload_type.value == 'url':
    url_response = requests.request("GET", upload_url.value)
    master_data['url_data'] = pd.read_csv(io.BytesIO(url_response.content))
else:
    if file_type.value == 'csv':
        tabs = None
        skiprows = 0
    else:
        tabs = [x.strip() for x in tab_names.value.split(',')]
        skiprows = skip_rows.value

    if subfolder.value == 'yes':
        subfolder = subfolder_name.value
    else:
        subfolder = None
    master_data = compile_raw_data(upload_filename.value, tabs, subfolder, delimiter_char = delimiter.value, skip_rows = skiprows, file_ext = file_type.value)


**Note:** If you see an AttributeError: 'NoneType' object has no attribute 'value' message above, simply rerun the last two code cells to reset the input parameters.

The following code cell will print out the attributes associated with the files you've uploaded and alert you of any errors:

In [9]:
for key, value in master_data.items():
    try:
        print('{} table was ingested with {} rows and {} columns'.format(key,value.shape[0],value.shape[1]))
    except:
        print('{} table failed to load'.format(key))

training_data table was ingested with 35000 rows and 18 columns


## 1.3 - Select a Data Frame

The following menus will allow you to select the dataset you would like to use in your modeling and the variables you would like included in the subsequent processes.  You can preview a sample of the data as well as increase or decrease the number of records returned by using the integer input widget (which has a default range; minimum rows = 1, maximum rows = 50).

Select an available frame from the list below:

In [10]:
dict_keys = widgets.Select(
    options=master_data.keys(),
    description='Tables:',
    disabled=False,
    layout=Layout(width='50%')
)

display(dict_keys)

Select(description='Tables:', layout=Layout(width='50%'), options=('training_data',), value='training_data')

## Reminder: Do not rerun the cell above after applying your inputs

### Click on and Run this cell to proceed

The cell below will provide a quick snapshot of the data you have selected above

In [11]:
master_data[dict_keys.value].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                35000 non-null  int64  
 1   name              35000 non-null  object 
 2   category          35000 non-null  object 
 3   main_category     35000 non-null  object 
 4   currency          35000 non-null  object 
 5   deadline          35000 non-null  object 
 6   goal              35000 non-null  float64
 7   launched          35000 non-null  object 
 8   pledged           35000 non-null  float64
 9   state             35000 non-null  object 
 10  backers           35000 non-null  int64  
 11  country           35000 non-null  object 
 12  usd_pledged       34659 non-null  float64
 13  usd_pledged_real  35000 non-null  float64
 14  usd_goal_real     35000 non-null  float64
 15  launch_month      35000 non-null  int64  
 16  launch_dow        35000 non-null  int64 

After selecting a frame above, select the variables you would like included in your workflow from the list below:

**NOTE:** To select multiple values from the picklist, either hold down the command key on your keyboard or click and hold the shift key to select ranges of variables.  You can scroll down if your mouse is within the widget window.

In [24]:
review_variables = widgets.SelectMultiple(
    options=master_data[dict_keys.value].columns.tolist(),
    description='Variables:',
    disabled=False,
    layout=Layout(width='50%')
)

display(review_variables)

SelectMultiple(description='Variables:', layout=Layout(width='50%'), options=('id', 'name', 'category', 'main_…

## Reminder: Do not rerun the cell above after applying your inputs

### Click on and Run this cell to proceed
Input the number of rows you'd like to sample:

In [30]:
review_var_list = []
for i in review_variables.value:
    review_var_list.append(i)
    
master_data['custom_table'] = master_data[dict_keys.value][review_var_list]

head_number = widgets.BoundedIntText(
    value=5,
    min=1,
    max=50,
    step=1,
    description='Rows:',
    disabled=False
)

def sample_view(head_number):
    sample = master_data['custom_table'].head(head_number)
    print(sample)

out = widgets.interactive_output(sample_view, {'head_number':head_number})

widgets.VBox([widgets.VBox([head_number]), out])

## 2.0 - Initiate H2O

The following cells will port your chosen dataframe to h2o and initiate a cluster.

## 2.1 - Select Your Target Variable

Your "Target" variable represents the thing you are attempting to predict. It should be either "categorical" (ex. text, labels) or "continuous" (ex. numeric values) in nature. The target and its type will impact which algorithms are used and the evaluation metrics that are useful in evaluating each models' performance.

Select your Target variable and note whether or not it is a categorical or continuous data type:

In [31]:
target = widgets.Select(
    options=master_data['custom_table'].columns.tolist(),
    description='Target',
    disabled=False
)

target_type = widgets.Select(
    options=['Continuous','Categorical'],
    description='Type',
    disabled=False,
)

display(target)
display(target_type)

Select(description='Target', options=('id', 'main_category', 'goal', 'state', 'country', 'launch_month', 'laun…

Select(description='Type', options=('Continuous', 'Categorical'), value='Continuous')

## Reminder: Do not rerun the cell above after applying your inputs

### Click on and Run this cell to proceed

## 2.2 - Initiate H2O

The code cell below will terminate any existing h2o instances and create a new instance

In [34]:
try:
    h2o.cluster().shutdown()
    h2o.init()
except:
    h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_181"; Java(TM) SE Runtime Environment (build 1.8.0_181-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from /anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/6k/j123vpgj42xf7z7vhxmc8bsw0000gn/T/tmpehw1zw5m
  JVM stdout: /var/folders/6k/j123vpgj42xf7z7vhxmc8bsw0000gn/T/tmpehw1zw5m/h2o_bblanchard006_started_from_python.out
  JVM stderr: /var/folders/6k/j123vpgj42xf7z7vhxmc8bsw0000gn/T/tmpehw1zw5m/h2o_bblanchard006_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.4
H2O cluster version age:,10 months and 3 days !!!
H2O cluster name:,H2O_from_python_bblanchard006_rd94b0
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


## IMPORTANT: If you are rerunning this workflow and have not "restarted your kernel" you will need to run the "cell" above up to three times to clear the instances.

**DO NOT PROCEED UNTIL** the above cell contains the following text (which will be visible just above a summary table):

Connecting to H2O server at http://127.0.0.1:54321 ... successful.

Load your dataset into h2o by running the command below

In [35]:
df = master_data['custom_table'].dropna(subset=[target.value])
df = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


Once the "Parse progress:" reaches 100% above, confirm that your dataset has been loaded correctly by reviewing the table below

In [36]:
df.describe()

Rows:35000
Cols:8




,id,main_category,goal,state,country,launch_month,launch_dow,duration
type,int,enum,real,enum,enum,int,int,int
mins,106144.0,,1.0,,,1.0,1.0,0.0
mean,991009165.4901404,,41407.85601142863,,,6.466428571428577,4.018514285714226,34.05877142857133
maxs,1177447262.0,,100000000.0,,,12.0,7.0,14867.0
sigma,299594360.51538,,880928.0793111619,,,3.3239116455511066,1.7024427916070275,80.32815423881766
zeros,0,,0,,,0,0,5
missing,0,0,0,0,0,0,0,0
0,1000002330.0,Publishing,1000.0,failed,GB,8.0,3.0,58.0
1,1000003930.0,Film & Video,30000.0,failed,US,9.0,7.0,60.0
2,1000004038.0,Film & Video,45000.0,failed,US,1.0,7.0,45.0


If your "Target" variable is categorical, the code below will convert it to a factor before modeling

In [37]:
if target_type.value == 'Categorical':
    df[target.value] = df[target.value].asfactor()

## 2.3 - Configure Models in H2O Flow

### In your browser, connecting to the H2O server at http://127.0.0.1:54321

After you have generated your model using H2O Flow and saved your model's h2o-genmodel.jar and MOJO files to your local machine, begin the next sequence of cells below.

## 3.0 - Score a New Dataset

The following process will walk you through uploading another dataset to score against your top model.

**Note:** The new dataset must contain the same fields that were used to train your models in the prior steps.  The structure of the new dataset does not have to be consistent with the one used in prior steps (ex. there is no need to align columns or remove fields).

## 3.1 - Upload Your Data (Excel and CSV files)

Follow the same process you used in subsequent steps to upload the dataset you would like to apply against your trained model.  This is the dataset with "unknown" values which your trained models will attempt to predict.

In [38]:
upload_type = widgets.RadioButtons(
    options=['local', 'url'],
    description='File Location:',
    disabled=False
)

upload_url = widgets.Text(
    value='https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv',
    placeholder='http://',
    description='URL:',
    disabled=False,
    layout=Layout(width='80%')
)
upload_filename = widgets.Text(
    value='hold_out_data.csv',
    placeholder='Sample File.csv',
    description='File Name:',
    disabled=False,
    layout=Layout(width='50%')
)

file_type = widgets.RadioButtons(
    options=['csv', 'xlsx'],
    description='File Type:',
    disabled=False
)

tab_names = widgets.Text(
    value='Sheet1, Sheet2, Sheet3, etc',
    placeholder='ALL EMPLOYEES, PAST EMPLOYEES',
    description='Tab(s):',
    disabled=False,
    layout=Layout(width='50%')
)

subfolder_name = widgets.Text(
    value='source_data',
    placeholder='Subfolder name',
    description='Subfolder:',
    disabled=False,
    layout=Layout(width='50%')
)

subfolder = widgets.RadioButtons(
    options=['no','yes'],
    value='no',
    description='Subfolder:',
    disabled=False
)

skip_rows = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Skip Rows:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

delimiter = widgets.RadioButtons(
    options=[',',';'],
    value=',',
    description='Delimiter:',
    disabled=False
)

def text_field(x):
    if(x=='xlsx'):
        display(tab_names)
        tab_names.on_submit(tab_names)
        display(skip_rows)
    else:
        display(delimiter)
        print('Tab Names: Not needed for csv files')

def sub_folder(y):
    if(y=='yes'):
        display(subfolder_name)
        subfolder_name.on_submit(subfolder_name)
    else:
        print('Using {} folder'.format(os.getcwd()))

def file_location(z):
    if(z=='local'):
        display(upload_filename)
        i = widgets.interactive(text_field, x=file_type)
        display(i)
        p = widgets.interactive(sub_folder, y=subfolder)
        display(p)
    else:
        display(upload_url)

q = widgets.interactive(file_location, z=upload_type)

display(q)

interactive(children=(RadioButtons(description='File Location:', options=('local', 'url'), value='local'), Out…

## Reminder: Do not rerun the cell above after applying your inputs

### Click on and Run this cell to proceed
The following code cell will attempt to ingest the data you've selected in the widgets above:

**Note About xlsx Files** - Depending on the number of tabs and the size of the data on each tab, ingesting an xlsx file can take several minutes to execute.  If possible, it may be more efficient to break your Excel file into separate csv files which take only a fraction of a second to ingest.

In [39]:
new_data = {}

if upload_type.value == 'url':
    url_response = requests.request("GET", upload_url.value)
    new_data['url_data'] = pd.read_csv(io.BytesIO(url_response.content))
else:
    if file_type.value == 'csv':
        tabs = None
        skiprows = 0
    else:
        tabs = [x.strip() for x in tab_names.value.split(',')]
        skiprows = skip_rows.value

    if subfolder.value == 'yes':
        subfolder = subfolder_name.value
    else:
        subfolder = None
    new_data = compile_raw_data(upload_filename.value, tabs, subfolder, delimiter_char = delimiter.value, skip_rows = skiprows, file_ext = file_type.value)


**Note:** If you see an AttributeError: 'NoneType' object has no attribute 'value' message above, simply rerun the last two code cells to reset the input parameters.

The following code cell will print out the attributes associated with the files you've uploaded and alert you of any errors:

In [40]:
for key, value in new_data.items():
    try:
        print('{} table was ingested with {} rows and {} columns'.format(key,value.shape[0],value.shape[1]))
    except:
        print('{} table failed to load'.format(key))

hold_out_data table was ingested with 15000 rows and 18 columns


## 3.2 - Select a Data Frame to be Scored

The following menus will allow you to select the dataset you would like to score against your trained model.  This dataset should contain the fields you used to train the models in prior steps, but it does not have to consistent of the same structure (ex. there is no need to remove unused columns or align column locations).

Select an available frame from the list below:

In [41]:
dict_keys = widgets.Select(
    options=new_data.keys(),
    description='Tables:',
    disabled=False,
    layout=Layout(width='50%')
)

display(dict_keys)

Select(description='Tables:', layout=Layout(width='50%'), options=('hold_out_data',), value='hold_out_data')

## Reminder: Do not rerun the cell above after applying your inputs

### Click on and Run this cell to proceed

**Select any date variables from your dataframe that need to be processed (H2O handles date fields very specifically)**

In [42]:
date_variables = widgets.SelectMultiple(
    options=new_data[dict_keys.value].columns.tolist(),
    description='Variables:',
    disabled=False
)

display(date_variables)

SelectMultiple(description='Variables:', options=('id', 'name', 'category', 'main_category', 'currency', 'dead…

## Reminder: Do not rerun the cell above after applying your inputs

### Click on and Run this cell to proceed

In [43]:
date_fields = [x for x in date_variables.value]

if len(date_fields)==0:
    print('Skipping date field processing; no variables selected')
else:
    for d in date_fields:
        try:
            new_data[dict_keys.value][d] = pd.to_datetime(new_data[dict_keys.value][d])
            new_data[dict_keys.value][d] = new_data[dict_keys.value][d].apply(lambda x: x.replace(tzinfo=timezone.utc).timestamp()).astype(int)
        except:
            print('Unable to convert {} to a date field'.format(d))


## 3.3 - Single or Batch Scoring

Select whether or you'd like to perform single-instance or batch-instance scoring:

In [44]:
pred_type = widgets.RadioButtons(
    options=['Single record', 'Batch records'],
    description='Score:',
    disabled=False
)

batch_instances = widgets.IntSlider(
    value=10,
    min=2,
    max=50,
    step=1,
    description='Batch Size:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def prediction_frame(x):
    if(x=='Batch records'):
        display(batch_instances)
    else:
        print('Single instance selected')
  
pred_instance = widgets.interactive(prediction_frame, x=pred_type)

display(pred_instance)

interactive(children=(RadioButtons(description='Score:', options=('Single record', 'Batch records'), value='Si…

In [45]:
if pred_type.value == 'Single record':
    instances = new_data[dict_keys.value].sample()
    get_keys = instances.columns.tolist()
    get_values = instances.values.tolist()

    df_dict = {get_keys[i]: get_values[0][i] for i in range(len(get_keys))} 
    df_dict = json.dumps(df_dict)
else:
    instances = new_data[dict_keys.value].head(batch_instances.value)
    get_keys = instances.columns.tolist()
    get_values = instances.values.tolist()

    dict_list = []
    for x in range(0,len(get_values)):
        df_dict = {get_keys[i]: get_values[x][i] for i in range(len(get_keys))}
        dict_list.append(df_dict)

    df_dict = json.dumps(dict_list)

Choose the zipped model file saved during the model building exercise in H2O Flow **(i.e., this is the MOJO file you downloaded)**

In [46]:
file_list = widgets.Select(
    options=os.listdir(),
    description='Files:',
    disabled=False,
    layout=Layout(width='100%')
)

display(file_list)

Select(description='Files:', layout=Layout(width='100%'), options=('.DS_Store', 'archive', 'training_data.csv'…

In [47]:
try:
    model_id = file_list.value.replace('.zip','')
except:
    print('Please choose the zipped file containing the model id')

Setup the scoring parameters.
**Note:** You do not have to change any of the parameters below:

In [48]:
gen_model_arg = os.getcwd() + os.sep + 'h2o-genmodel.jar'
best_model_id = model_id # Change the model id if you build your own
mojo_model_args = os.getcwd() + os.sep + best_model_id + '.zip'
h2o_predictor_class = 'water.util.H2OPredictor'
json_data = str(df_dict)

Score the new instances against the saved model and return the prediction results

In [49]:
output = subprocess.check_output(['java' , '-Xmx4g', '-cp', gen_model_arg, h2o_predictor_class,mojo_model_args, json_data], shell=False).decode()   

# Format the results in a dataframe for additional analysis
if pred_type.value == 'Single record':
    pf = pd.read_json(output, orient='index')
else:
    pf = pd.read_json(output, orient='record')


View the predicted values for your instance(s):

In [50]:
pf

,labelIndex,label,classProbabilities
0,3,successful,"[0.061620989023153, 0.326686199528232, 0.00072..."
1,1,failed,"[0.088079156575999, 0.6546569156541711, 0.0007..."
2,1,failed,"[0.09491996702034801, 0.462752264713559, 0.000..."
3,1,failed,"[0.087470593604413, 0.624300565412503, 0.00097..."
4,1,failed,"[0.198709362110981, 0.658420763849431, 0.00087..."
5,1,failed,"[0.169101948636207, 0.739902424551629, 0.00168..."
6,1,failed,"[0.09227236518715701, 0.6221993440918401, 0.00..."
7,1,failed,"[0.11118028030966001, 0.6583856934378041, 0.00..."
8,1,failed,"[0.053858366785661006, 0.673290680746043, 0.00..."
9,3,successful,"[0.032201695411663005, 0.19140933937970603, 0...."


View the original instance(s):

In [51]:
instances

,id,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real,launch_month,launch_dow,duration
0,1177451632,The Everyday Project,Experimental,Film & Video,USD,1403136000,50.0,1400627280,223.0,successful,35,US,223.00,223.00,50.00,5,3,29
1,1177454326,The Creation of a Digital Fashion Magazine!,Periodicals,Publishing,USD,1334361600,5000.0,1331829540,0.0,failed,0,US,0.00,0.00,5000.00,3,5,29
2,117745590,Aerial Dream: A collision of snowboarding and RC,Film & Video,Film & Video,USD,1352937600,7500.0,1348601700,35.0,failed,2,US,35.00,35.00,7500.00,9,3,50
3,1177459197,"Ipuina Kontatu, (a feature length documentary)",Documentary,Film & Video,USD,1279929600,12000.0,1275683340,12205.0,successful,59,US,12205.00,12205.00,12000.00,6,6,49
4,1177460347,Bring The RG to Reading | Art for everyone. (C...,Public Art,Art,GBP,1405814400,50000.0,1402413540,5907.0,canceled,31,GB,9927.31,10034.83,84940.12,6,3,39
5,1177470526,Cow Invader,Video Games,Games,EUR,1447977600,20000.0,1442857500,13.0,failed,2,FR,14.69,13.89,21375.97,9,2,59
6,1177480984,"It's a Photo Van, man!",Photography,Photography,USD,1383955200,9000.0,1380133080,2730.0,failed,44,US,2730.00,2730.00,9000.00,9,4,44
7,1177489610,NOVA Fashion Week Designer Project,Fashion,Fashion,USD,1347062400,10000.0,1344480840,1150.0,failed,3,US,1150.00,1150.00,10000.00,8,5,30
8,1177500682,Fresh Start Juice Co.,Drinks,Food,USD,1424736000,7000.0,1420861740,2.0,failed,2,US,2.00,2.00,7000.00,1,7,45
9,1177507060,THE HOTEL COLORS,Theater,Theater,USD,1365811200,5500.0,1364142600,6673.0,successful,126,US,6673.00,6673.00,5500.00,3,1,19


#### If you need any support,  please feel free to contact me at bablanchard@smu.edu